In [ ]:
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os.path
import re
from pykalman import KalmanFilter
import matplotlib.pyplot as plt
import time
import math
from munkres import Munkres,print_matrix
import logging

###for logging
def mylogger(name):
    logger = logging.getLogger(name)
    print(len(logger.handlers))
    if not len(logger.handlers):
        print(len(logger.handlers))
        logger = logging.getLogger(name)
        hdlr = logging.FileHandler('log/tracking_kalman_hungarian.log')
        formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
        hdlr.setFormatter(formatter)
        logger.addHandler(hdlr)
        logger.setLevel(logging.INFO)

    return logger

my_logger = mylogger('log_tracking_kalman')

##Kalman + Hungarian algorithm
##using Gaussian bg subtraction and medianBlur
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows = True)
fgbg.setShadowValue(0)
#fgbg.setVarThreshold(20)
fgbg.setShadowThreshold(0.5)

#imagedir = "image_track_0620/"
#savedir = "track_kalman_0620/"
imagedir = "image_track_0705/"
savedir = "track_kalman_0705/"
#imagedir = "sample_avi/"
#savedir = "sample_avi_track/"
#savedir_blur = "sample_avi_track_bg/"
#imagedir = "csula/csula_student_1/"
#savedir = "csula/csula_student_1_detect/"
#savedir_blur = "csula/csula_student_1_bg/"

##Configuration for  threasholds
##threashold for decide if x and y and their predictions are the same
POSITION_THREASHOLD= 30

##missing frame numbers to dismiss the object from the current_tracks
MISSING_THREASHOLD = 10

##minimum contour area to consider
CONTOUR_THREASHOLD_MIN = 500
##maximum contour area to consider
CONTOUR_THREASHOLD_MAX = 8000

COST_THREASHOLD = 80

##pedestrian counter
COUNTER_m = 0
COUNTER_p = 0
OVERLAP_THREASHOLD = 0.30
DISTANCE_THREASHOLD = 20

##init array to hold current objects
current_tracks = []

##debug
debug = True

#Pedestrians = []
Pedestrians ={}

class MovingObject(object):
    def __init__(self,id,position):
        self.id = id
        #self.position = ([position])
        #self.predicted_position = ([position])
        self.position = np.array([position])
        self.predicted_position = np.array([position])
        self.frames_since_seen = 0
        self.frame_since_start = 0
        self.counted = 0
        self.next_mean = []
        self.next_covariance = []
        self.detection = 0
        
    def count_frames(self):
        self.frame_since_start = self.frame_since_start + 1
        return self.frame_since_start
    
    def add_position(self, position_new):
        #self.position.append(position_new)
        self.position = np.append(self.position, position_new, axis=0)
        self.frames_since_seen = 0
        
    def add_predicted_position(self, next_position):
        #self.predicted_position.append(next_position)
        self.predicted_position = np.append(self.predicted_position, next_position, axis=0)
        
    def init_kalman_filter(self):
        transition_matrix=[[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]]
        observation_matrix=[[1,0,0,0],[0,1,0,0]]
        initstate = [0,0,0,0]
        initial_state_covariance = [[2,0,0,0],[0,2,0,0],[0,0,0.5,0],[0,0,0,0.5]]
        ## value of Q 
        transition_covariance = [[5,0,0,0],[0,5,0,0],[0,0,2,0],[0,0,0,0.5]]
        ##Value of R
        #observation_covariance = [[5,0,0,0],[0,5,0,0],[0,0,2,0],[0,0,2,0]]
        observation_covariance =[[5,0],[0,5]]
        self.kf = KalmanFilter(transition_matrices = transition_matrix,
                  observation_matrices = observation_matrix,
                  initial_state_mean = initstate,
                  initial_state_covariance = initial_state_covariance,
                  transition_covariance = transition_covariance,
                  observation_covariance= observation_covariance
                  #em_vars=['transition_covariance', 'initial_state_covariance']
                )
    
    def kalman_update(self, new_position):
        next_mean = self.next_mean
        next_covariance = self.next_covariance
        next_mean, next_covariance = self.kf.filter_update(
            filtered_state_mean = next_mean, 
            filtered_state_covariance = next_covariance,
            observation = new_position)
        self.set_next_mean(next_mean)
        self.set_next_covariance(next_covariance)
        self.add_predicted_position([self.next_mean[:2]])
        self.frames_since_seen = 0
    
    def set_next_mean(self,mean):
        self.next_mean = mean
    def set_next_covariance(self,covariance):
        self.next_covariance = covariance
    
    def set_frames_since_seen(self, num):
        frames_since_seen = num
        
    def detection_increase(self):
        self.detection += 1


def track_new_object(position, tracks, counter):
    new_mObject = MovingObject(counter,position)
    new_mObject.add_position([position])
    if debug:
        print("create new object, id: ", new_mObject.id, "current: ", new_mObject.position, "predicted: ", new_mObject.predicted_position)

    new_mObject.init_kalman_filter()
    filtered_state_means, filtered_state_covariances = new_mObject.kf.filter(new_mObject.position)
    new_mObject.set_next_mean(filtered_state_means[-1])
    new_mObject.set_next_covariance(filtered_state_covariances[-1])

    ##add to current_tracks
    tracks.append(new_mObject)
    
#function overlap calculate how much 2 rectangles overlap
def overlap_area(boxes):
    if(len(boxes) == 0):
        return 0
    
    xx1 = max(boxes[0,0], boxes[1,0])
    yy1 = max(boxes[0,1], boxes[1,1])
    xx2 = min(boxes[0,2], boxes[1,2]) 
    yy2 = min(boxes[0,3], boxes[1,3])
    
    w = max(0, xx2 - xx1 + 1)
    h = max(0, yy2 - yy1 + 1)
    
    #print(boxes[0,2], boxes[1,2], xx2 , xx1)
    #print(w)
    ##box1 area
    area1 = (boxes[0,2]-boxes[0,0]) * (boxes[0,3]-boxes[0,1])
    ##box2 area
    area2 = (boxes[1,2]-boxes[1,0]) * (boxes[1,3]-boxes[1,1])
    if area1 > area2:
        area = area2
    else:
        area = area1
        
    overlap = (w * h) / area
    
    return overlap

##calculate distance between 2 points, pos: [0,0], points: [[1,1],[2,2],[3,3]]
def get_costs(pos,points):
    #distances = np.array([[math.sqrt((x2-pos[0])**2+(y2-pos[1])**2)] for (x2,y2) in points])
    distances = [math.floor(math.sqrt((x2-pos[0])**2+(y2-pos[1])**2)) for (x2,y2) in points]
    return distances

##Main program starts here
n = 0

##hog for pedestrian detection
hog = cv2.HOGDescriptor((48,96), (16,16), (8,8), (8,8), 9)
hog.setSVMDetector(cv2.HOGDescriptor_getDaimlerPeopleDetector())
hogParams = {'winStride': (8, 8), 'padding': (16, 16), 'scale': 1.05}

##Loop through each frame and find the contours
##then:
## 
##  if no movingObject existing yet, init the first one
##    run Kalman filter to predict next position
##  else, in the following frame:
##    save all the detected contours corrdinations and calculate the cost to form a matrix
##      --> distance from each assigned movingObject to the detected coutours in this frame
##    use Hungarian algorithm to calculate assignment
##      in case: more contours than detected object, means unassigned moving object exists
##      find the index and start a new movingObject and track it
##      another case: tracked movingObjects index not shown in the assignment, means it's not updated in this frame
##    Use the assignment to update Kalman filter
my_logger.info("Start detection...")
if (not os.path.exists(imagedir)):
    print(imagedir + "does not exist! Please provide valid directory. Exiting...")
    
for dirName, subdirList, fileList in os.walk(imagedir):
    for fname in fileList:
        ##get file number from filename, ex. image12630.jpg -> 12630
        if(re.search(r".jpg",fname)):
            filepath = dirName + fname
            print(filepath)
            
            ##read in image
            frame = cv2.imread(filepath)
            #if debug:
            my_logger.info(fname + "****************************************************")
            ##copy
            orig = frame.copy()
            
            ##remove background and find contour
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            fgmask = fgbg.apply(gray)
            th = cv2.threshold(fgmask.copy(), 245, 255, cv2.THRESH_BINARY)[1]
            
            #kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
            #erode = cv2.erode(th,kernel,iterations = 1)
            #dilate = cv2.dilate(erode,kernel,iterations = 1)
            #opened = cv2.morphologyEx(dilate, cv2.MORPH_OPEN, kernel)
            #blurred = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)
            #blurred = cv2.medianBlur(closed,3)
            ##for debug, write blurred image into frame
            #frame = cv2.medianBlur(th,3)
            blurred = cv2.medianBlur(th,1)
            
            ##detect pedestrian
            ##run detection against blurred image
            (ped_rects, weights) = hog.detectMultiScale(frame, **hogParams)
            
            ##find contour in background removed frames
            (_, cnts, _) = cv2.findContours(blurred, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            
            ##mark pedestrians with write bounding box in backgroud
            #if debug:
                #for (ped_x, ped_y, ped_w, ped_h) in ped_rects:
                    #cv2.rectangle(blurred, (ped_x, ped_y), (ped_x + 40, ped_y + 70), (255, 0, 0), 2)
                    #print("pedestrian detected", ped_x, ped_y, ped_x + 50, ped_y + 70)
               
            ##debug bg removed files
            #blurred_file_path = savedir_blur + fname+ "blur.jpg"
            #cv2.imwrite(blurred_file_path, blurred)
            
            if(len(current_tracks) > 0):
                for index, obj in enumerate(current_tracks):
                    obj.counted = 0
            
            ##loop through all detected contours and save them into array contours
            contours = np.zeros((0,2))
            contours_orig = np.zeros((0,4))
            for ct in cnts:
                ##skip contours too small or too large
                #if(cv2.contourArea(ct) < 500 and cv2.contourArea(ct) > 100):
                    #print("Contour area: ", cv2.contourArea(ct))
                if cv2.contourArea(ct) < CONTOUR_THREASHOLD_MIN or cv2.contourArea(ct) > CONTOUR_THREASHOLD_MAX:
                    continue
                (x, y, w, h) = cv2.boundingRect(ct)
                #cv2.rectangle(frame, (x, y), (x + w, y + h), (240, 0, 159), 2)
                ##center of the bounding rectangle
                cx = x + int(w/2)
                cy = y + int(h/2)
                #log_string = "contour %d %d %d %d %d %d" % (x, y, w, h, cx,cy)
                #my_logger.info(log_string)
                ##threashold  to exclude cars
                #if(w > 3*h):
                    #if debug:
                        #print("skip ct w > 3h", x, y, w, h)
                    #continue
                
                ##draw rectangle around contour
                cv2.rectangle(frame, (x, y), (x + w, y + h), (240, 0, 159), 2)
                cv2.rectangle(blurred, (x, y), (x + w, y + h), (240, 0, 159), 2)
                #rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
                ##location for Kalman filter to track
                tdata = [cx,cy]
                #if debug:
                    #print("new location ",tdata, "-----------------------------------")
                    #s = "new location [{0},{1}]"
                    #log_string = s.format(*tdata)
                    #my_logger.info(log_string)
                ##add positions to contours array
                contours_orig = np.append(contours_orig,[[x,y,w,h]],axis=0)
                contours = np.append(contours, [tdata],axis=0)
            ##done with saving all the detected contours
                
            #############################################
            ##For first frame, add all detected contours as new MovingObjects
            if(len(current_tracks) == 0):
                for cont in contours:
                    
                    COUNTER_m += 1
                    ##create new movingObject
                    new_mObject = MovingObject(COUNTER_m,tdata)
                    new_mObject.add_position([tdata])
                    if debug:
                        print("create first object, id: ", new_mObject.id, "current: ", new_mObject.position, "predicted: ", new_mObject.predicted_position)
                    my_logger.info("create first object, id: %d ", new_mObject.id)
                    my_logger.info("predicted position")
                    my_logger.info(new_mObject.predicted_position)
                    
                    new_mObject.init_kalman_filter()
                    filtered_state_means, filtered_state_covariances = new_mObject.kf.filter(new_mObject.position)
                    new_mObject.set_next_mean(filtered_state_means[-1])
                    new_mObject.set_next_covariance(filtered_state_covariances[-1])

                    ##add to current_tracks
                    current_tracks.append(new_mObject)
                    #print("COUNTER_m++", COUNTER_m)
                    my_logger.info("COUNTER_m++ %d", COUNTER_m)

            ##from the 2nd frame, calculate cost using predicted position and new contour positions
            else:
                ##save all the positions to a matrix and calculate the cost
                ##initiate a matrix for calculating assianment by Hungarian algorithm
                if(len(contours) == 0):
                    print("No contour found!")
                    savepath = savedir + fname
                    cv2.imwrite(savepath, frame)
                    n = n + 1
                    continue
                    
                #matrix_h = np.zeros((0,len(contours)))
                matrix_h =[]
                remove_obj = []
                if debug:
                    print("number of current objects", len(current_tracks))

                ##loop through existing tracked movingObjects
                ## find movingObjects which are disappeared already and add to remove_obj array
                if debug:
                    #print("current tracks")
                    for track in current_tracks:
                        my_logger.info("current track id %d:", track.id)
                        my_logger.info(track.predicted_position[-1])
                        #print(track.predicted_position[-1])
                    #print("contours arrays :", contours)
                    my_logger.info("contours array: ")
                    my_logger.info(contours)
                    
                #for index, obj in enumerate(current_tracks):
                    ##if a moving object hasn't been updated for 10 frames then remove it
                    #if obj.frames_since_seen > MISSING_THREASHOLD:
                        #remove_obj.append(obj)
                        #if debug:
                            #my_logger.info("%d not updates for 10 frames, add to removing list", obj.id)
                        #continue
               
                ## caclulate the cost to each contour and form a matrix
                ## use the available_tracks list to remove tracking object which has distance larger than threshold
                ## so it won't mess up the rest
                available_tracks = []
                for index, obj in enumerate(current_tracks):
                    ##calculate costs for each tracked movingObjects using their predicted position
                    costs = get_costs(obj.predicted_position[-1], contours)
                    #print("costs",costs)
                    my_logger.info("costs")
                    my_logger.info(costs)
                    
                    ## if tracking object to all contours distances are too large, then not to consider it at all
                    if all(c > COST_THREASHOLD for c in costs):
                        ##update it with KF predicted position
                        obj.kalman_update(obj.predicted_position[-1])
                        my_logger.info("disappeard object id %d", obj.id)
                        my_logger.info(obj.predicted_position[-1])
                        continue
                    matrix_h.append(costs)
                    available_tracks.append(obj)
                my_logger.info("matrix")
                my_logger.info(matrix_h)
                    
                    
                ## matrix_h: 
                ##
                ##       |contour1 | contour2 |...
                ##-----------------------------------
                ## mObj1 | cost11   | cost12  |...
                ## mObj2 | cost21   | cost22  |...
                ##
                ##calculate assignment with the matrix_h
                ## a missing column means new track
                ## a missing row means missing track in the frame
                
                munkres = Munkres()
                indexes = munkres.compute(matrix_h)
                #print("indexes", indexes)
                my_logger.info("indexes")
                my_logger.info(indexes)
                
                #if debug:
                total = 0
                for row, column in indexes:
                    value = matrix_h[row][column]
                    total += value
                    #print ('(%d, %d) -> %d' % (row, column, value))
                    #print ('total cost: %d' % total)
                    
                ## next : loop through the indexes got from the assignment and update Kalman filter
                ## find untracked MovingObjects and track them
                ## the movingObjects not being updated, set last_since_seen += 1
                
                ## loop through the contours and update Kalman filter for each contour
                indexes_np = np.array(indexes)
                for index_c,cont in enumerate(contours):
                    ## found contour index, then update this contour position with the tracked object
                    if index_c in indexes_np[:,1]:
                        contour_index_list = indexes_np[:,1].tolist()
                        ##find index of the movinbObject
                        ## ex. [(0, 2), (1, 1), (3, 0), (4, 3)] --> 2nd element with contour[0], index_m=2
                        index_m = contour_index_list.index(index_c)
                        
                        ##find index in current_tracks
                        index_track = indexes_np[index_m,0]
                        #print("m_contour_list",contour_index_list, "(", index_track,",",index_c,")")
                
                        ##check if cost is bigger than threashold then track it as a new one
                        #print("index in matrix ", index_track,index_c)
                        if matrix_h[index_track][index_c] > COST_THREASHOLD:
                            print("Too much distance in between, cannot be the same object", matrix_h[index_track][index_c])
                            my_logger.info("Too much distance in between, cannot be the same object")
                            ##create new MovingObject of this contour
                            COUNTER_m += 1
                            track_new_object(cont, current_tracks, COUNTER_m)
                            #print("COUNTER_m", COUNTER_m)
                            continue
                
                        ##get the object from the index and update it's Kalman filter
                        ##obj_m = current_tracks[index_track]
                        if debug:
                            my_logger.info("available_tracks")
                            for a_tracks in available_tracks:
                                my_logger.info(a_tracks.id)
                        
                        obj_m = available_tracks[index_track]
                        ##get corresponding contour position
                        position_new = contours[index_c]
                        #print("get new position to update KF", obj_m.predicted_position, position_new)
                        my_logger.info("object id: %d,", obj_m.id)
                        my_logger.info("contour index: %d,", index_c)
                        my_logger.info("predicted position")
                        my_logger.info(obj_m.predicted_position)
                        my_logger.info("New position used to update KF")
                        my_logger.info(position_new)
                        obj_m.kalman_update(position_new)
                        
                        obj_m.counted = 1
                        ## mark the moving object with the id
                        prx = position_new[0]
                        pry = position_new[1]
                        #label_points = np.array([[[position_new[0], position_new[1]],[position_new[0]+40, position_new[1]],[position_new[0]+40, position_new[1]-20], [position_new[0], position_new[1]-20]]])
                        #cv2.fillPoly(frame,label_points,(0,255,0))
                        cv2.putText(frame,str(obj_m.id),(int(position_new[0]),int(position_new[1]-20)),cv2.FONT_HERSHEY_SIMPLEX,0.8, (255,0,0),2)
                        
                        ##get the original contour x,y,w,h, not the center corrdination
                        cont_x,cont_y,cont_w,cont_h = contours_orig[index_c]
                        
                        ##add pedestrian detection
                        for (prx,pry,prw,prh) in ped_rects:
                            c_x = prx + int(prw/2)
                            c_y = pry + int(prh/2)
                            
                            ##compare by distance between center points
                            """distance = math.sqrt((c_x-position_new[0])**2+(c_y-position_new[1])**2)
                            if debug:
                                    print("distance ", distance);
                            if distance < DISTANCE_THREASHOLD:
                            """
                            
                            ##compare by bounding box overlap
                            boxes_2compare = np.array([[cont_x,cont_y,cont_x+cont_w,cont_y+cont_h],[prx,pry,prx+40,pry+70]])
                            #if debug:
                                #print("compare ped rects: ", prx,pry,prw,prh)
                                
                            o_rate = overlap_area(boxes_2compare)
                            if(o_rate > OVERLAP_THREASHOLD):
                                obj_m.detection_increase()
                                print("Pedestrian detected by overlapping", obj_m.detection)
                                
                                ##mark overlapped pedestrian and moving object in blue box
                                #if debug:
                                    #print("close enough to the pedestrian detection: ", c_x,c_y,prx,pry)
                                    #cv2.rectangle(frame, (prx, pry), (prx + 40, pry + 70), (0, 0, 255), 2)
                                    
                                ##for pedestrian detection
                                if(obj_m.detection >= 3):
                                    ##check if this is a new pedestrian we detected by checking
                                    ##if the object id is in Pedestrians dict or not
                                    if (not obj_m.id in Pedestrians.keys()):
                                        #Pedestrians.append(obj_m.id)
                                        COUNTER_p += 1
                                        Pedestrians[obj_m.id] = COUNTER_p
                                        print("No. ", COUNTER_p , "Pedestrians" + str(obj_m.id))

                                    ##if added already then draw notation in the frame
                                    else:
                                        ##mark the pedestrian in the image
                                        pr_points = np.array([[[prx, pry],[prx+prw, pry],[prx+40, pry-20], [prx, pry-20]]])
                                        #cv2.fillPoly(frame,pr_points,(0,255,0))
                                        #cv2.putText(frame,str(Pedestrians[obj_m.id]),(int((prx+w)/2),int(pry-5)),cv2.FONT_HERSHEY_SIMPLEX,0.5, (255,0,0))
                    
                    ## not found in columns, means not being tracked, so start tracking it
                    else:
                        print("index_c", index_c)
                        position_new = contours[index_c]
                        COUNTER_m += 1
                        new_mObject = MovingObject(COUNTER_m,position_new)
                        new_mObject.add_position([position_new])
                        new_mObject.init_kalman_filter()
                        if debug:
                            #print("create new object, id: ", new_mObject.id, "current: ", new_mObject.position, "predicted: ", new_mObject.predicted_position)
                            my_logger.info("create new object, id: %d", new_mObject.id)
                            my_logger.info("predicted position")
                            my_logger.info(new_mObject.predicted_position)
                            
                        filtered_state_means, filtered_state_covariances = new_mObject.kf.filter(new_mObject.position)
                        new_mObject.set_next_mean(filtered_state_means[-1])
                        new_mObject.set_next_covariance(filtered_state_covariances[-1])
                    
                        ##add to current_tracks
                        current_tracks.append(new_mObject)
                        cv2.putText(frame,str(new_mObject.id),(int(position_new[0]),int(position_new[1]-20)),cv2.FONT_HERSHEY_SIMPLEX,0.8, (255,0,0),2)
                        #print("COUNTER_m++", COUNTER_m)
                        my_logger.info("counter %d", COUNTER_m)
                        
                ##these are tracks missed either because they disappeared 
                ## or because they are temporarily invisable 
                for index,obj in enumerate(available_tracks):
                    if index not in indexes_np[:,0]:
                        ## not update in this frame, increase frames_since_seen
                        obj.frames_since_seen += 1
                        ##but we update KF with predicted location
                        obj.kalman_update(obj.predicted_position[-1])
                        my_logger.info("disappeard object id %d", obj.id)
                        my_logger.info(obj.predicted_position[-1])
                    
                ##remove movingObj not updated for more than threasholds numbers of frames  
                for index, obj in enumerate(current_tracks):
                    ##if a moving object hasn't been updated for 10 frames then remove it
                    if obj.frames_since_seen > MISSING_THREASHOLD:
                        remove_obj.append(obj)
                        if debug:
                            my_logger.info("%d not updates for 10 frames, add to removing list", obj.id)
                        continue
                
                ##remove movingObj not updated for more than threasholds numbers of frames  
                for index, obj in enumerate(current_tracks):
                    if obj in remove_obj:
                        if debug:
                            print("remove", obj.id, obj.position)
                        del current_tracks[index]
                    
                    
            ##debug bg removed files
            #blurred_file_path = savedir_blur + fname+ "blur.jpg"
            #cv2.imwrite(blurred_file_path, blurred)
            
            savepath = savedir + fname
            cv2.imwrite(savepath, frame)
            n = n + 1
            
print("Counted " , COUNTER_p , " pedestrians")